In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import os
import json
import cv2 as cv
from tqdm import tqdm

def IOU(a, b):
    assert a.shape == b.shape and len(a.shape) == 2
    return np.count_nonzero(np.logical_and(a, b)) / np.count_nonzero(np.logical_or(a, b))


In [ ]:
# VOC labels
label_colors= np.asarray(
        [
            [0, 0, 0],
            [128, 0, 0],
            [0, 128, 0],
            [128, 128, 0],
            [0, 0, 128],
            [128, 0, 128],
            [0, 128, 128],
            [128, 128, 128],
            [64, 0, 0],
            [192, 0, 0],
            [64, 128, 0],
            [192, 128, 0],
            [64, 0, 128],
            [192, 0, 128],
            [64, 128, 128],
            [192, 128, 128],
            [0, 64, 0],
            [128, 64, 0],
            [0, 192, 0],
            [128, 192, 0],
            [0, 64, 128],
        ]
    ).astype(np.uint8)

label_texts = [
    'background',
    'aeroplane', 
    'bicycle', 
    'bird',
    'boat',
    'bottle',
    'bus',
    'car',
    'cat',
    'chair',
    'cow',
    'diningtable',
    'dog',
    'horse',
    'motorbike',
    'person',
    'potted plant',
    'sheep',
    'sofa',
    'train',
    'tv/monitor'
]

In [ ]:
pred = '/nfs/ws1/ryan/CleanCode/Projects/Peekaboo/Experiments/Dreams/untracked/for_xiang/ref_voc'
prefix = '/nfs/ws1/datasets/RefVOC/'
gt = '/raid/datasets/pascal_voc/VOC2012/SegmentationClass'

from IPython.display import display
from PIL import Image

def zip_to_single_channel(l):
    return (l[...,0] // 4)  + (l[...,1] // 16) + l[...,2] // 64

zipped_labels = zip_to_single_channel(label_colors).tolist()
print(zipped_labels)

displayed=5

def eval_peekaboo_voc(prefix, pred, gt):
    unfinished = []
    with open(os.path.join(prefix, 'cropped.txt'), 'r') as f:
        names = f.readlines()
    
    # random.shuffle(names)
    
    res = []
    
    folders = glob(f'{pred}/*')
    methods = [folder.split('/')[-1] for folder in folders]
    
    for name in tqdm(names):
        words = name.split()
        image_name = words[0]
        cls_name = ' '.join(words[1:])
        
        with open(f'{prefix}/cropped-{image_name}.jpg.txt', 'r') as f:
            x1, x2, y1, y2 = f.readline().split()
            x1 = int(x1)
            x2 = int(x2)
            y1 = int(y1)
            y2 = int(y2)
            
        
            
            gt_img = cv.imread(os.path.join(gt, f'{image_name}.png'))[y1:y2, x1:x2, ::-1]
            gt_img = zip_to_single_channel(gt_img)
            h, w = gt_img.shape
            cls_idx = label_texts.index(cls_name)
            cls_color = zipped_labels[cls_idx]

            mapb = gt_img == cls_color

            for method, folder in zip(methods, folders):
                if method == 'preview_images':
                    continue
                image_target = os.path.join(folder, f'alpha', f'cropped-{image_name}.jpg')
                if not os.path.exists(image_target):
                    unfinished.append(image_target)
                    continue
                pred_img = cv.resize(cv.imread(image_target, 0), (w, h), cv.INTER_NEAREST)
                
                import rp
                
#                 alpha=pred_img
#                 std=alpha.std()
#                 alpha=alpha-alpha.mean()*.9
#                 alpha=alpha/std/1.5
#                 pred_img=alpha
                
#                 R=55
#                 pred_img=rp.cv_dilate(pred_img,R,circular=True)
#                 # pred_img=rp.cv_erode(pred_img,R,circular=True)
#                 pred_img=rp.cv_erode(pred_img,R,circular=True)
#                 # R=10
#                 # pred_img=rp.cv_erode(pred_img,R,circular=True)
#                 # pred_img=rp.cv_dilate(pred_img,R,circular=True)
#                 # pred_img=rp.cv_dilate(pred_img,20,circular=True)
                
#                 # alpha=pred_img
#                 # std=alpha.std()
#                 # alpha=alpha-alpha.mean()*.9
#                 # alpha=alpha/std/2
#                 # pred_img=alpha
                alpha=pred_img
                std=alpha.std()
                alpha=alpha-alpha.mean()*.55
                alpha=alpha/std/2
                pred_img=alpha
                
                R=55
                pred_img=rp.cv_dilate(pred_img,R,circular=True)
                # pred_img=rp.cv_erode(pred_img,R,circular=True)
                pred_img=rp.cv_erode(pred_img,R,circular=True)
                # R=10
                # pred_img=rp.cv_erode(pred_img,R,circular=True)
                # pred_img=rp.cv_dilate(pred_img,R,circular=True)
                # pred_img=rp.cv_dilate(pred_img,20,circular=True)
                
                # alpha=pred_img
                # std=alpha.std()
                # alpha=alpha-alpha.mean()*.9
                # alpha=alpha/std/2
                # pred_img=alpha

                pred_img=rp.as_byte_image(pred_img)
                
                global displayed
                if displayed:
                    rp.display_image(pred_img)
                    displayed-=1
                # print(pred_img)
                # 1/0
                
                for t in range(25):
                    thre=t*10
                    # thre = t * 255 / 25
                    mapa = pred_img > thre
                    # print(thre)

                    # print(mapa.shape, mapb.shape)
                    iou = IOU(mapa, mapb)
                    """
                    # debug
                    print(image_name, cls_name, iou)
                    display(Image.fromarray(mapa))
                    display(Image.fromarray(mapb))
                    break
                    """
                    res.append((iou, cls_name, f'Peekaboo-{method}', thre))
    print("Unfinished images:")
    print(len(unfinished))
    return pd.DataFrame(data=res, columns=['IoU', 'class', 'method', 'threshold']), unfinished
gp1, unfinished1 = eval_peekaboo_voc(prefix, pred, gt)

In [ ]:
pd.set_option('display.max_rows', 100)  
gp1

In [ ]:
pd.set_option('display.max_rows', None)  
gp1.groupby(['method', 'threshold', 'class']).mean().round(3)

In [ ]:
pd.set_option('display.max_rows', None)  
gp1.groupby(['method', 'threshold']).mean().round(5)

In [ ]:
gp1.to_pickle("./RefVOC-Peekaboo.pkl")  

In [ ]:
prefix = '/nfs/ws1/datasets/RefVOC-MO/'
pred = '/nfs/ws1/ryan/CleanCode/Projects/Peekaboo/Experiments/Dreams/untracked/for_xiang/ref_voc_mo'
gt = '/raid/datasets/pascal_voc/VOC2012/SegmentationClass'

gp2, unfinished2 = eval_peekaboo_voc(prefix, pred, gt)

In [ ]:
gp2.groupby(['method', 'class', 'threshold']).mean().round(3)

In [ ]:
gp2.groupby(['method', 'threshold']).mean().round(5)

In [ ]:
gp2.to_pickle("./RefVOC-MO-Peekaboo.pkl")  

In [ ]:
def eval_peekaboo_coco(prefix, pred, gt):
    unfinished = []
    with open(os.path.join(prefix, 'cropped.txt'), 'r') as f:
        names = f.readlines()
    with open(os.path.join(prefix, 'map.txt'), 'r') as f:
        maps = f.readlines()
    res = []
    # random.shuffle(names)
    folders = glob(f'{pred}/*')
    methods = [folder.split('/')[-1] for folder in folders]
    
    for i, name in zip(tqdm(maps), names):
        i = int(i)
        words = name.split()
        image_name = words[0]
        png_name = image_name.replace('jpg', 'png')
        cls_name = ' '.join(words[1:])
        
        bbox = f'{prefix}/{image_name}.txt'
        if os.path.exists(bbox):
            with open(bbox, 'r') as f:
                x1, x2, y1, y2 = f.readline().split()
                x1 = int(x1)
                x2 = int(x2)
                y1 = int(y1)
                y2 = int(y2)
            gt_img = cv.imread(os.path.join(gt, f'{i}-{png_name}'), 0)[y1:y2, x1:x2]
        else:
            gt_img = cv.imread(os.path.join(gt, f'{i}-{png_name}'), 0)
            
        h, w = gt_img.shape
        
        mapb = gt_img > 0
        
        for method, folder in zip(methods, folders):
            if method == 'preview_images':
                continue
            image_target = os.path.join(folder, f'alpha', image_name)
            if not os.path.exists(image_target):
                unfinished.append(image_target)
                continue
            pred_img = cv.resize(cv.imread(image_target, 0), (w, h), cv.INTER_NEAREST)

            for t in range(10):
                thre = t * 255 / 10
                mapa = pred_img > thre

                # print(mapa.shape, mapb.shape)
                iou = IOU(mapa, mapb)
                """
                # debug
                print(image_name, cls_name, iou)
                display(Image.fromarray(mapa))
                display(Image.fromarray(mapb))
                break
                """
                res.append((iou, cls_name, f'Peekaboo-{method}', thre))

    print("Unfinished images:")
    print(unfinished)
    return pd.DataFrame(data=res, columns=['IoU', 'class', 'method', 'threshold']), unfinished

In [ ]:
prefix = '/nfs/ws1/datasets/RefCOCO/'
pred = '/nfs/ws1/ryan/CleanCode/Projects/Peekaboo/Experiments/Dreams/untracked/for_xiang/coco'
gt = '/nfs/ws1/datasets/RefCOCO/label/'
gp3, unfinished3 = eval_peekaboo_coco(prefix, pred, gt)